In [1]:
import numpy as np
from PIL import Image
from numpy import asarray
from paz.pipelines import MiniXceptionFER
from paz.backend.image import show_image

In [2]:
emotions = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

In [13]:
image = Image.open("./data/images/4/Happy.jpg")
image.show()
numpy_image = asarray(image)

classify = MiniXceptionFER()
predictions = classify(numpy_image)

scores = predictions["scores"].flatten()

for emotion, score in zip(emotions, scores):
    print(f"{emotion}: {score:.4f}")

print(f"\nPredicted Emotion: {predictions['class_name'].upper()}")

print(predictions)

angry: 0.1127
disgust: 0.0130
fear: 0.0243
happy: 0.6239
sad: 0.0582
surprise: 0.0278
neutral: 0.1401

Predicted Emotion: HAPPY
{'class_name': 'happy', 'scores': array([[0.11265942, 0.01297694, 0.02432909, 0.62394935, 0.05821391,
        0.02779427, 0.14007702]], dtype=float32)}


In [7]:
from paz.applications import HaarCascadeFrontalFace, MiniXceptionFER
import paz.processors as pr

class EmotionDetector(pr.Processor):
    def __init__(self):
        super(EmotionDetector, self).__init__()
        self.detect = HaarCascadeFrontalFace(draw=False)
        self.crop = pr.CropBoxes2D()
        self.classify = MiniXceptionFER()
        self.draw = pr.DrawBoxes2D(self.classify.class_names)

    def call(self, image):
        boxes2D = self.detect(image)['boxes2D']
        cropped_images = self.crop(image, boxes2D)
        for cropped_image, box2D in zip(cropped_images, boxes2D):
            box2D.class_name = self.classify(cropped_image)['class_name']
        return self.draw(image, boxes2D)

In [8]:
detect = EmotionDetector()

predictions = detect(numpy_image.copy())

Image.fromarray(predictions).show()

test accuracy for dataset,
note that the datset is to our knowledge not scientifically pre-classified.

We note that some emotions should be classified differently in our opinion.

Create Confusion Matrix

create script to take picture with camera